## _*Using Qiskit Aqua for graph partition problems*_

Here we consider a simplified graph partition problem.
Consider a graph $G = (V, E)$, where $V$ denotes the set of n vertices and $E$ the set of edges. 
The objective of graph partition is to partition $G$ into two sets of the same size (let us assume we have even number of vertices), 
while minimizing the capacity of the edges across the two sets.

We will go through three examples to show:
1. How to run the optimization using the declarative approach
2. How to run the optimization using the programmatic approach
3. How how to run the optimization with the VQE.

Note the objective_value below is defined as the the number of crossing edges. The goal is to minimize this value.


#### The problem and the brute-force method

The graph involved in our example is as follows. The graph is in the adjacent matrix form.

In [1]:
import numpy as np
from qiskit import BasicAer
from qiskit.aqua import run_algorithm
from qiskit.aqua.input import EnergyInput
from qiskit.aqua.translators.ising import graph_partition
from qiskit.aqua.algorithms import ExactEigensolver

np.random.seed(100)
num_nodes = 4
w = graph_partition.random_graph(num_nodes, edge_prob=0.8, weight_range=10)
print(w)

[[ 0.  4.  5.  3.]
 [ 4.  0. -5.  7.]
 [ 5. -5.  0.  0.]
 [ 3.  7.  0.  0.]]


The brute-force method is as follows. Basically, we exhaustively try all the binary assignments. In each binary assignment, the entry of a vertex is either 0 (meaning the vertex is in the first partition) or 1 (meaning the vertex is in the second partition). We print the binary assignment that satisfies the definition of the graph partition and corresponds to the minimial number of crossing edges.

In [2]:
def brute_force():
    # use the brute-force way to generate the oracle
    def bitfield(n, L):
        result = np.binary_repr(n, L)
        return [int(digit) for digit in result]  # [2:] to chop off the "0b" part

    L = num_nodes
    max = 2**L
    minimal_v = np.inf
    for i in range(max):
        cur = bitfield(i, L)

        how_many_nonzero = np.count_nonzero(cur)
        if how_many_nonzero * 2 != L:  # not balanced
            continue

        cur_v = graph_partition.objective_value(np.array(cur), w)
        if cur_v < minimal_v:
            minimal_v = cur_v
    return minimal_v

sol = brute_force()
print("Objective value computed by the brute-force method is", sol)

Objective value computed by the brute-force method is 3


#### Part I: Run the optimization using the declarative approach

In [3]:
qubit_op, offset = graph_partition.get_graph_partition_qubitops(w)
algo_input = EnergyInput(qubit_op)

params = {
    'problem': {'name': 'ising'},
    'algorithm': {'name': 'ExactEigensolver'}
}
result = run_algorithm(params, algo_input)

x = graph_partition.sample_most_likely(result['eigvecs'][0])
# check against the oracle
ising_sol = graph_partition.get_graph_solution(x)
np.testing.assert_array_equal(ising_sol, [0, 1, 0, 1])
print("Objective value computed by ExactEigensolver is", graph_partition.objective_value(x, w))

Objective value computed by ExactEigensolver is 3.0


#### Part II: Run the optimization using the programmatic approach

In [4]:
algo = ExactEigensolver(algo_input.qubit_op, k=1, aux_operators=[])
result = algo.run()

x = graph_partition.sample_most_likely(result['eigvecs'][0])
# check against the oracle
ising_sol = graph_partition.get_graph_solution(x)
np.testing.assert_array_equal(ising_sol, [0, 1, 0, 1])
print("Objective value computed by the ExactEigensolver is", graph_partition.objective_value(x, w))

Objective value computed by the ExactEigensolver is 3.0


#### Part III: Run the optimization with the VQE

##### Declarative

We reuse the EnergyInput object we created above. VQE algorithm needs an optimizer and a variational form. Then also we need a quantum backend on which the algorithm will run.

In [5]:
params = {
    'problem': {'name': 'ising', 'random_seed': 10598},
    'algorithm': {'name': 'VQE'},
    'optimizer': {'name': 'COBYLA'},
    'variational_form': {'name': 'RY', 'depth': 5, 'entanglement': 'linear'}
}
backend = BasicAer.get_backend('statevector_simulator')
result = run_algorithm(params, algo_input, backend=backend)

x = graph_partition.sample_most_likely(result['eigvecs'][0])
# check against the oracle
ising_sol = graph_partition.get_graph_solution(x)
np.testing.assert_array_equal(ising_sol, [1, 0, 1, 0])
print("Objective value computed by VQE is", graph_partition.objective_value(x, w))

Objective value computed by VQE is 3.0


##### Programmatic

We can create the objects directly ourselves too and run VQE for the result.

In [6]:
from qiskit.aqua import aqua_globals
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.aqua.components.variational_forms import RY

aqua_globals.random_seed = 10598

optimizer = COBYLA()
var_form = RY(qubit_op.num_qubits, depth=5, entanglement='linear')
vqe = VQE(qubit_op, var_form, optimizer)

backend = BasicAer.get_backend('statevector_simulator')
result = vqe.run(backend)

x = graph_partition.sample_most_likely(result['eigvecs'][0])
# check against the oracle
ising_sol = graph_partition.get_graph_solution(x)
np.testing.assert_array_equal(ising_sol, [1, 0, 1, 0])
print("Objective value computed by VQE is", graph_partition.objective_value(x, w))

Objective value computed by VQE is 3.0
